In [1]:
import os
import sys
sys.path.insert(0, r'../../../quetzal')
from quetzal.os.parallel_call import parallel_call_notebook
import itertools
import pandas as pd
if not os.path.exists('log/'):
    os.makedirs('log/')

In [2]:
kwargs = {'workers':6, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}


In [3]:
params = {'catchment_radius': {'bus': '500', 'subway': '800', 'rail': '1000'}}
scenarios=['montreal', 'montreal_2', 'montreal_3', 'jakarta', 'auckland', 'vancouver']

montreal: pt, road and population <br>
montreal_2: pt and population <br>
montreal_3: only pt

In [222]:
arg_list = []
for scen in scenarios:
    path = f'../../scenarios/{scen}'.format(scen=scen)
    arg = str({'training_folder': path, 'params':params}).replace("'",'"')
    arg_list.append(arg)

delete all outputs

In [223]:
for scen in scenarios:
    path = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    files = os.listdir(path)
    for file in files:
        os.remove(path+file)

In [224]:
parallel_call_notebook('main.ipynb', arg_list=arg_list, **kwargs)

[NbConvertApp] Converting notebook main.ipynb to python
[NbConvertApp] Writing 27403 bytes to main.py


0 main.py {"training_folder": "../../scenarios/montreal", "params": {"catchment_radius": {"bus": "500", "subway": "800", "rail": "1000"}}}
1 main.py {"training_folder": "../../scenarios/montreal_2", "params": {"catchment_radius": {"bus": "500", "subway": "800", "rail": "1000"}}}
2 main.py {"training_folder": "../../scenarios/montreal_3", "params": {"catchment_radius": {"bus": "500", "subway": "800", "rail": "1000"}}}
93 seconds


In [225]:
expected_files = ['route_type_metrics.csv','population_mesh.geojson','route_id_metrics.csv','nodes.geojson']
for scen in scenarios:
    path = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    files = os.listdir(path)
    for file in expected_files:
        assert file in files


In [226]:
# check there is no NaN in csv
# if no population.geojson dont check catchment as it could be NaN (its ok)
for scen in scenarios:
    path = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    test = pd.read_csv(path+'route_type_metrics.csv')
    if 'population.geojson' not in os.listdir(f'../../scenarios/{scen}/inputs/'.format(scen=scen)):
        test = test.drop(columns=['catchment'])
    assert not test.isnull().any().any()


In [227]:
# check there is no NaN in csv
# if no population.geojson dont check catchment as it could be NaN (its ok)
for scen in scenarios[0:1]:
    path = f'../../scenarios/{scen}/outputs/'.format(scen=scen)
    test = pd.read_csv(path+'route_id_metrics.csv')
    if 'population.geojson' not in os.listdir(f'../../scenarios/{scen}/inputs/'.format(scen=scen)):
        test = test.drop(columns=['catchment'])
    assert not test.isnull().any().any()
